In [1]:
# Require Pandas version >= 2.2.2
import pandas as pd
assert pd.__version__ >= '2.2.2', 'Please update Pandas to version 2.2.2 or newer'
import pandas as pd, os, pyreadr
import warnings
import time
import numpy as  np
import pandas as pd
import pandas as pd
import visualization as viz
import data_loading as dl
import risk_pricing as rp
import statsmodels.api as sm
import statsmodels.formula.api as smf
import importlib
importlib.reload(rp)
importlib.reload(viz)
importlib.reload(dl)
warnings.filterwarnings("ignore")
start_time = time.time()
source_folder_path = "/Users/pedrovallocci/Documents/PhD (local)/Research/Github/KnowledgeKRisk_10Ks/src/"

In [2]:
# # Creating additional models
# import matplotlib.pyplot as plt
# embeddings = Embeddings()
# EmbeddingsHKRModel.plot_cosine_similarity(["intellectual property", "patent", "knowledge capital", "research and development", "key talent"])
# Save last plot:
# plt.savefig("cosine_similarity.png")
# base_path = "/Users/pedrovallocci/Documents/PhD (local)/Research/Github/KnowledgeKRisk_10Ks/text/"
# dir_path = os.path.join(base_path)
# os.makedirs(dir_path, exist_ok=True)
# figfolder = os.path.join(dir_path, "")
# # Save to the folder:
# plt.savefig(figfolder + "cosine_similarity_terms.png")
# EmbeddingsHKRModel().from_topic_similarity(embeddings, "emb_knocap", "knowledge capital", to_csv = True)
# EmbeddingsHKRModel().from_topic_similarity(embeddings, "emb_patents", "patents", to_csv = True)
# EmbeddingsHKRModel().from_topic_similarity(embeddings, "emb_intcap", "intangible capital", to_csv = True)

In [3]:
import risk_pricing as rp
import importlib
import embedding_tools as et
import visualization as viz
from embedding_tools import Embeddings
from embedding_tools import EmbeddingsHKRModel
importlib.reload(rp)
importlib.reload(viz)
quantiles = 10
# embeddings = Embeddings()
# EmbeddingsHKRModel().from_topic_similarity(embeddings, "emb_kt", "key talent", to_csv = True)
# cases_list = [("embeddings_km10_ipcs", "pfkki3me3mb", 3)]
cases_list = [("embeddings_km10_ipcs", "pfkki3me3mb", 3),
              ("embeddings_km10_ipcs", "pfkki3me3mb", 10),
              ("dicfullmc10thr10defnob40noa0_8_hdp", "pfkk2me3mb", 5),
              ("dicfullmc10thr10defnob40noa0_8_4t", "pfkk2me3mb", 5),
              ("embeddings_km10_ipcs", "pfkk2me3mb", 3),
              ("embeddings_km10_ipcs", "pfkk2me3mb", 10), 
              ("emb_patents", "pfkki3me3mb", 3)
             ]

for modelname, pfname, quantiles in cases_list:
    figfolder, add_innerkk_pf, cuts = viz.setup(quantiles, modelname, pfname)
    amazon_nov01_short, cequity_mapper, ff3fw, ff5fw, ff3fm, ff5fm, topic_map_unlabeled, comparison_measures, stoxmo_orig, stoxda_orig, stoxwe_orig = \
        dl.load_dataframes(modelname, start_time, clean_again = False)
    topic_map = viz.label_topic_map(topic_map_unlabeled, modelname, cuts = cuts)
    fmb_5, fmb_3, fmb_1, df_betas5, _, df_betas1, eret_we5, _, stoxwe_add, eret_we_pct5, eret_we_pct3 = rp.famaMacBethMultiCase(pfname, add_innerkk_pf, cuts, cequity_mapper, ff3fw, ff5fw, stoxwe_orig, topic_map, log_returns = False)
    viz.explore_topic_map(topic_map, figfolder)
    viz.explore_fmb([fmb_1, fmb_3, fmb_5], figfolder)
    viz.explore_eret_we(eret_we5, eret_we_pct5, figfolder)
    viz.tex_compare_kk_measures(comparison_measures, figfolder)
    viz.plot_returns(stoxwe_add, figfolder)
    # # viz.amazon_graph(amazon_nov01_short, figfolder)
    # # viz.filecounter(figfolder)
    viz.explore_stoxda(stoxda_orig, cequity_mapper, topic_map, figfolder)
    viz.explore_betas(df_betas5, quantiles, figfolder)

Running model embeddings_km10_ipcs with 3 quantiles and pfkki3me3mb portfolio
Executing label_topic_map...
Running Fama-MacBeth regressions: 5 factors


/Users/pedrovallocci/Documents/PhD (local)/Research/Github/KnowledgeKRisk_10Ks/src/risk_pricing/risk_pricing.py:59: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
/Users/pedrovallocci/Documents/PhD (local)/Research/Github/KnowledgeKRisk_10Ks/src/risk_pricing/risk_pricing.py:85: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
/Users/pedrovallocci/Documents/PhD (local)/Research/Github/KnowledgeKRisk_10Ks/src/risk_pr

Finished processing stoxwe
New kernel
Finished Fama-MacBeth
Running Fama-MacBeth regressions: 3 factors


/Users/pedrovallocci/Documents/PhD (local)/Research/Github/KnowledgeKRisk_10Ks/src/risk_pricing/risk_pricing.py:59: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
/Users/pedrovallocci/Documents/PhD (local)/Research/Github/KnowledgeKRisk_10Ks/src/risk_pricing/risk_pricing.py:85: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
/Users/pedrovallocci/Documents/PhD (local)/Research/Github/KnowledgeKRisk_10Ks/src/risk_pr

Finished processing stoxwe
New kernel
Finished Fama-MacBeth
New kernel
Running Fama-MacBeth regressions: 1 factor
Executing explore_topic_map...
Executing tex_calculate_correlation_matrix...
Executing tex_descriptive_statistics...
Executing fig_mean_tiy...
Executing tex_average_topic_loadings_by_high_tech...
Executing tex_sample_topic_loadings...
Executing df_skill_correlations...
Executing df_patent_correlations...
Executing fig_share_dominant_kk_by_ind...
Executing fig_heatmap_topicvskkpt...


...

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

In [ ]:
viz.explore_eret_we(eret_we5, figfolder)

Executing fig_h1b_vs_smb_kkhml...
Executing tex_HKR_vs_mktrf...
Now using pseudo-monthly returns
Finished using pseudo-monthly returns!
Executing tex_fmb_results_statistics...
Iteration: 0, Objective: 253.86368947417273
Iteration: 10, Objective: 100.40313599506032
Iteration: 20, Objective: 88.58850933868358
Iteration: 30, Objective: 83.69958530953906
Iteration: 40, Objective: 79.34870189978307
Iteration: 50, Objective: 74.6849017167029
Iteration: 60, Objective: 64.97023304028227
         Current function value: 64.537077
         Iterations: 69
         Function evaluations: 6974
         Gradient evaluations: 118
Iteration: 0, Objective: 64.67623883353177
Iteration: 10, Objective: 60.598034204437816
Iteration: 20, Objective: 59.998711898764334
Iteration: 30, Objective: 59.584872780982096
Iteration: 40, Objective: 59.38008098173363
Iteration: 50, Objective: 59.19037213129217
Iteration: 60, Objective: 58.67256759953228
         Current function value: 58.672562
         Iterations: 67
 

<Figure size 640x480 with 0 Axes>